# Installation

0. Tested on Windows
1. Download and install Anaconda
2. Download fastai fork https://github.com/n-waves/fastai/releases/tag/poleval2018 and unzip
3. Open Anaconda Prompt as administrator
4. ```bash
cd UNZIP_DIR/fastai-poleval2018
conda env update
conda activate fastai
conda install pytorch=0.4.1 -c pytorch
pip install sentencepiece
pip install torchtext==0.2.3
jupyter notebook
```







In [17]:
import sys
!{sys.executable} -m pip install sentencepiece
!{sys.executable} -m pip install torchtext==0.2.3
!conda list --prefix {sys.prefix}

# packages in environment at C:\ProgramData\Anaconda3\envs\fastai:
#
# Name                    Version                   Build  Channel
asn1crypto                0.24.0                   py36_0  
atomicwrites              1.3.0                    py36_1  
attrs                     19.1.0                   py36_1  
backcall                  0.1.0                    py36_0  
backports                 1.0                      py36_1  
backports.os              0.1.1                    py36_0  
bcolz                     1.2.1            py36h830ac7b_0  
beautifulsoup4            4.6.3                    pypi_0    pypi
blas                      1.0                         mkl  
bleach                    3.1.0                    py36_0  
blosc                     1.16.3               h7bd577a_0  
bokeh                     1.2.0                    py36_0  
bzip2                     1.0.6                hfa6e2cd_5  
ca-certificates           2019.1.23                     0  
certifi           

pandoc                    2.2.3.2                       0  
pandocfilters             1.4.2                    py36_1  
parso                     0.4.0                      py_0  
partd                     0.3.10                   py36_1  
path.py                   12.0.1                     py_0  
patsy                     0.5.1                    py36_0  
pbr                       5.2.1                    pypi_0    pypi
pcre                      8.43                 ha925a31_0  
pexpect                   4.7.0                    py36_0  
pickleshare               0.7.5                    py36_0  
pillow                    6.0.0            py36hdc69c19_0  
pip                       19.1.1                   py36_0  
plac                      0.9.6                    py36_0  
plotnine                  0.5.1                    pypi_0    pypi
pluggy                    0.11.0                     py_0  
preshed                   2.0.1            py36h33f27b4_0  
prettytable               0.

In [2]:
from fastai.text import *

In [3]:
h5_filepath = "../work/up_low50k/models/fwd_v50k_finetune_lm_enc.h5"
model_filepath = "../work/up_low50k/tmp/sp-50k.model"

In [4]:
import sentencepiece as spm
spm_processor = spm.SentencePieceProcessor()
spm_processor.Load(model_filepath)


spm_processor.SetEncodeExtraOptions("bos:eos")
spm_processor.SetDecodeExtraOptions("bos:eos")

True

In [5]:
def get_lm(bptt, max_seq, n_tok, emb_sz, n_hid, n_layers, pad_token, bidir=False,
           tie_weights=True, qrnn=False):
    rnn_enc = MultiBatchRNN(bptt, max_seq, n_tok, emb_sz, n_hid, n_layers, pad_token=pad_token, bidir=bidir, qrnn=qrnn)
    enc = rnn_enc.encoder if tie_weights else None
    return SequentialRNN(rnn_enc, LinearDecoder(n_tok, emb_sz, 0, tie_encoder=enc))

In [18]:
print(torch.cuda.device_count())

torch.cuda.device(0)
torch.cuda.current_device()


1


0

In [7]:
UNK_ID = 0
PAD_ID = 1
BOS_ID = 2
EOS_ID = 3
UP_ID  = 4
bs=22

em_sz,nh,nl = 400 ,1150,4

bptt=5
vs = len(spm_processor)

lm = get_lm(bptt, 1000000, vs, em_sz, nh, nl, PAD_ID)
lm = to_gpu(lm)
load_model(lm[0],h5_filepath)
lm.reset()
lm.eval()


SequentialRNN(
  (0): MultiBatchRNN(
    (encoder): Embedding(50000, 400, padding_idx=1)
    (encoder_with_dropout): EmbeddingDropout(
      (embed): Embedding(50000, 400, padding_idx=1)
    )
    (rnns): ModuleList(
      (0): WeightDrop(
        (module): LSTM(400, 1150)
      )
      (1): WeightDrop(
        (module): LSTM(1150, 1150)
      )
      (2): WeightDrop(
        (module): LSTM(1150, 1150)
      )
      (3): WeightDrop(
        (module): LSTM(1150, 400)
      )
    )
    (dropouti): LockedDropout()
    (dropouths): ModuleList(
      (0): LockedDropout()
      (1): LockedDropout()
      (2): LockedDropout()
      (3): LockedDropout()
    )
  )
  (1): LinearDecoder(
    (decoder): Linear(in_features=400, out_features=50000, bias=False)
    (dropout): LockedDropout()
  )
)

In [8]:

class LMTextDataset(Dataset):
    def __init__(self, x):
        self.x = x

    def __getitem__(self, idx):
        sentence = self.x[idx]
        return sentence[:-1], sentence[1:]

    def __len__(self):
        return len(self.x)

In [9]:
def next_tokens(ids_, model, deterministic, omit_ids=[]):
    ids = [np.array(ids_)]
    test_ds = LMTextDataset(ids)
    test_samp = SortSampler(ids, key=lambda x: len(ids[x]))
    dl = DataLoader(test_ds,
                    bs,
                    transpose=True,
                    transpose_y=True,
                    num_workers=1,
                    pad_idx=PAD_ID,
                    sampler=test_samp,
                    pre_pad=False)

    tensor1 = None
    with no_grad_context():
        for (x, y) in dl:
            tensor1 = model(x)
    p = tensor1[0]

    arg = p[-1]
    r = int(
        torch.argmax(arg) if deterministic else torch.
        multinomial(p[-1].exp(), 1)) # probability is in logharitm

    while r in omit_ids + [ids_[-1]]:  #, BOS_ID,EOS_ID, UNK_ID]:
        arg[r] = -1
        r = int(torch.argmax(arg))

    predicted_ids = [r]
    return predicted_ids


In [11]:

def next_words_best(ss, lm, n_words, deterministic=True, omit_ids=[]):
    ss_ids = spm_processor.encode_as_ids(ss)
    wip = ss
    wip_ids = ss_ids
    for i in range(n_words):
        next_ = next_tokens(wip_ids, lm, deterministic, omit_ids=omit_ids)
        wip_ids = wip_ids + next_
        wip = spm_processor.decode_ids(wip_ids)
        wip_ids = spm_processor.encode_as_ids(wip)

    return wip

In [12]:
sentences = [
    "Warszawa to największe", "Te zabawki należą do",
    "Policjant przygląda się", "Na środku skrzyżowania widać",
    "Właściciel samochodu widział złodzieja z",
    "Prezydent z premierem rozmawiali wczoraj o", "Witaj drogi",
    "Gdybym wiedział wtedy dokładnie to co wiem teraz, to bym się nie",
    "Gdybym wiedziała wtedy dokładnie to co wiem teraz, to bym się nie",
    "Polscy naukowcy odkryli w Tatrach nowy gatunek istoty żywej. Zwięrzę to przypomina małpę, lecz porusza się na dwóch nogach i potrafi posługiwać się narzędziami. Przy dłuższej obserwacji okazało się, że potrafi również posługiwać się językiem polskim, a konkretnie gwarą podhalańską. Zwierzę to zostało nazwane"
]

In [13]:
def prediction(sentence: str, n_words=70, deterministic=False):
    return next_words_best(sentence, lm, n_words, deterministic=deterministic)

In [16]:
for sentence in sentences:
    print("")
    print(prediction(sentence, deterministic=False))


Warszawa to największe miasto posiadające finansowo następnym razem zaplecze biznesowo-poziomowe ... isar-S Columbia , a zaczyna się przeznaczeniem Łużd współczesnej sandomierskiej Anoiny , star

Te zabawki należą do ulubionych fizycznie najmłodszym użytkownikiem wzrosło ustawienie panelowego podawane na minutę do czasu 10.00 .. zł , opisane jest czasświąteczne towarzystwo loterii fantowej .. Kurcze będzie

Policjant przygląda się najbliższej lekcji , uwierzy wynajmujących wKT postronnym i dotkniętym niepowodzeniem skomplikowanym czynnościom patrolowym sformułowanym przez jego rodziców , Libertymax Recife widocznymmencie samochodowym zekryj nasze wolne sekrety odzwierciedlały prawdziwe wybitny średniconizm bohaterstwa respektowania mądrości wyszukanych postaw doczesnych wdzięczni enginealiści , spowodowało jednakowoż nieetycznego jej wyszkolenia

Na środku skrzyżowania widać ciemne odgłosy Rekreacji Kardiologicznej i Oceanii Pacanciu ,wdiecznego Dantego uwalniania czci isłuchania najm

In [15]:
for sentence in sentences:
    print("")
    print(prediction(sentence, deterministic=True))


Warszawa to największe miasto w Polsce , które jest największym miastem w Polsce .. 

Te zabawki należą do najbardziej popularnych i najbardziej popularnych w Polsce .. 

Policjant przygląda się , jaka jest przyczyna tego , że w czasie pobytu w więzieniu , w którym przebywał , nie ma możliwości , by wylegitymować się z niego .. 

Na środku skrzyżowania widać ślady po 

Właściciel samochodu widział złodzieja z 

Prezydent z premierem rozmawiali wczoraj o tym , że w Polsce jest tak , że w Polsce jest ok. 50 tys. osób , a w Polsce jest ok. 300 tys. osób .. , a w Polsce jest ok. 300 tys. osób .. , a w Polsce jest ok. 300 tys. osób .. , a w Polsce jest ok. 300 tys. osób ..

Witaj drogi , nie 

Gdybym wiedział wtedy dokładnie to co wiem teraz, to bym się nie spodziewał , że to nie jest tak , że nie ma co się bać .. 

Gdybym wiedziała wtedy dokładnie to co wiem teraz, to bym się nie spodziewała że to będzie jakiś żart .. 

Polscy naukowcy odkryli w Tatrach nowy gatunek istoty żywej. Zwięrzę 